In [9]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor
from llm import LLMInteractor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
interactor = SlackInteractor()
data = interactor.fetch_new_messages()

Set conversations_oldest to 1722399936.894109
Loaded 25 old messages
Found 2 new messages
Updated complete_conversations.pkl with new messages


In [55]:
threads = interactor.organize_threads(data)

In [56]:
interactor.get_conversation_stats(data)

,max_len,total_length,total_messages,average_message_size
user_name,,,,
Eli.chen,20,37,2,18.5


In [57]:
threads[0]

{'channel': 'general',
 'thread_ts': Timestamp('2024-08-02 20:54:23.551868916'),
 'messages': [{'ts': Timestamp('2024-08-02 20:54:23.551868916'),
   'user': 'Eli.chen',
   'text': "what's a burrito?",
   'is_bot': False,
   'minutes_ago': 0}]}

In [45]:
from pandas import Timestamp

llm_interactor = LLMInteractor()
thread = {'channel': 'general',
  'thread_ts': Timestamp('2024-08-01 23:18:53.991878986'),
  'messages': [{'ts': Timestamp('2024-08-01 23:18:53.991878986'),
    'user': 'Eli.chen',
    'text': 'hello everyone. my name is Eli',
    'is_bot': False,
    'minutes_ago': 153}]}
# thread = threads[0]
guideline_prompt = "Please provide a helpful and concise response to the conversation above."
response = llm_interactor.generate_thread_response(thread, guideline_prompt)
response

"Hello Eli! It's nice to meet you. How can I assist you today? Feel free to ask any questions or let me know if there's anything specific you'd like to discuss."

In [58]:
for thread in threads:
    if not thread['messages'][-1]['is_bot']:
        response = llm_interactor.generate_thread_response(thread, guideline_prompt)
        interactor.post_thread_reply(thread, response)

Posted reply to thread 1722632063.551869 in channel general
Posted reply to thread 1722632067.855989 in channel general
